In [1]:
import pandas as pd

import json
import glob
import os
import re

from shared import *

In [2]:
base = "../exp/"
paths = glob.glob(base + "data/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': path_to_graph(run['args'][1]),
    'metric': run['live_weight_file'],
    'queries': run['queries'],
    'epsilon': run['epsilon'],
} for run in data for algo in run['algo_runs']])

In [51]:
queries['instance'] = queries['graph'].map({'OSM Europe': 'OSM Eur ', 'DIMACS Europe': 'DIMACS Eur ', 'OSM Germany': 'OSM Ger '}) + queries['metric'].map({'fake_traffic': 'Syn', 'heavy_traffic': 'Fri', 'lite_traffic': 'Tue'})

In [54]:
queries_sub = queries.query("~(graph == 'OSM Germany' & metric == 'fake_traffic') & epsilon == 0.2")
table = queries_sub.groupby(['queries', 'instance', 'algo']) \
    .mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'running_time_ms', 'failed']] \
    .unstack()

table['failed'] *= 100
table = table.round(1)
# for col in ['running_time_ms', 'num_forbidden_paths']:
#     table[col] = table[col].round(1)
# for col in ['length_increase_percent', 'num_iterations']:
#     table[col] = table[col].round(2)
table = table.rename(columns={ 'iterative_detour_blocking': 'IDB', 'iterative_path_blocking': 'IPB' })

# table = table.swaplevel(axis='columns').sort_index(axis='columns').reindex(columns=['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'running_time_ms', 'failed'], level=1)

output = table.to_latex()
output = output.replace('queries/1h', R"\multirow{4}{*}{\rotatebox[origin=c]{90}{1h}}")
output = output.replace('queries/4h', R"\addlinespace \multirow{4}{*}{\rotatebox[origin=c]{90}{4h}}")
output = output.replace('queries/uniform', R"\addlinespace \multirow{4}{*}{\rotatebox[origin=c]{90}{Random}}")
lines = output.split("\n")
lines = [R'\setlength{\tabcolsep}{4pt}'] + lines[:2] + [
    R" & & \multicolumn{2}{c}{Increase $[\%]$} & \multicolumn{2}{c}{Iterations} & \multicolumn{2}{c}{Blocked Paths} & \multicolumn{2}{c}{Time [ms]} & \multicolumn{2}{c}{Failed $[\%]$} \\",
    R"\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6} \cmidrule(l{3pt}r{3pt}){7-8} \cmidrule(l{3pt}r{3pt}){9-10} \cmidrule(l{3pt}r{3pt}){11-12}",
    R" & & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB \\"
] + lines[5:]
output = add_latex_big_number_spaces("\n".join(lines) + "\n")

print(output)
table

\setlength{\tabcolsep}{4pt}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 & & \multicolumn{2}{c}{Increase $[\%]$} & \multicolumn{2}{c}{Iterations} & \multicolumn{2}{c}{Blocked Paths} & \multicolumn{2}{c}{Time [ms]} & \multicolumn{2}{c}{Failed $[\%]$} \\
\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6} \cmidrule(l{3pt}r{3pt}){7-8} \cmidrule(l{3pt}r{3pt}){9-10} \cmidrule(l{3pt}r{3pt}){11-12}
 & & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB \\
\midrule
\multirow{4}{*}{\rotatebox[origin=c]{90}{1h}} & DIMACS Eur Syn &                     3.6 &  1.0 &            6.9 &   33.0 &                24.5 &   98.8 &           108.4 &    29.4 &    1.6 &  10.5 \\
                & OSM Eur Syn &                     0.3 &  0.2 &            1.8 &   11.0 &                 1.4 &   39.7 &             3.7 &     9.0 &    0.2 &   2.2 \\
                & OSM Ger Fri &                     1.7 &  0.3 &            8.3 &   48.8 &                44.2 &  123.3 &           191.5 &    71.6 &    1.1 &  

length_increase_percent      num_iterations  \
algo                                               IDB  IPB            IDB   
queries         instance                                                     
queries/1h      DIMACS Eur Syn                     3.6  1.0            6.9   
                OSM Eur Syn                        0.3  0.2            1.8   
                OSM Ger Fri                        1.7  0.3            8.3   
                OSM Ger Tue                        0.3  0.1            1.7   
queries/4h      DIMACS Eur Syn                     3.8  1.2           16.2   
                OSM Eur Syn                        0.3  0.2            3.1   
                OSM Ger Fri                        3.3  0.3           51.5   
                OSM Ger Tue                        0.4  0.1            4.3   
queries/uniform DIMACS Eur Syn                     3.5  1.1           45.6   
                OSM Eur Syn                        0.3  0.2           11.5   
                OSM Ger Fri                        3.1  0.3           47.9   
                OSM Ger Tue                        0.4  0.1            4.1   

                                      num_forbidden_paths         \
algo                              IPB                 IDB    IPB   
queries         instance                                           
queries/1h      DIMACS Eur Syn   33.0                24.5   98.8   
                OSM Eur Syn      11.0                 1.4   39.7   
                OSM Ger Fri      48.8                44.2  123.3   
                OSM Ger Tue      13.6                 4.4   33.8   
queries/4h      DIMACS Eur Syn  115.3                75.0  479.5   
                OSM Eur Syn      36.6                 4.8  155.2   
                OSM Ger Fri     179.2               407.4  531.8   
                OSM Ger Tue      49.7                23.0  147.0   
queries/uniform DIMACS Eur Syn  189.3               222.2  909.7   
                OSM Eur Syn     140.3                30.1  875.5   
                OSM Ger Fri     158.3               386.7  469.0   
                OSM Ger Tue      45.4                24.5  126.7   

                               running_time_ms         failed        
algo                                       IDB     IPB    IDB   IPB  
queries         instance                                             
queries/1h      DIMACS Eur Syn           108.4    29.4    1.6  10.5  
                OSM Eur Syn                3.7     9.0    0.2   2.2  
                OSM Ger Fri              191.5    71.6    1.1  17.7  
                OSM Ger Tue               10.8    16.7    0.0   3.9  
queries/4h      DIMACS Eur Syn           354.6   301.3    3.1  41.1  
                OSM Eur Syn               28.2    68.5    0.6   7.3  
                OSM Ger Fri             6631.5  1000.9    8.4  69.8  
                OSM Ger Tue              105.3   141.5    0.0  17.4  
queries/uniform DIMACS Eur Syn         10515.9  7656.5   11.8  71.6  
                OSM Eur Syn              662.3  9713.8    3.1  43.2  
                OSM Ger Fri             9567.2   926.0    8.8  61.7  
                OSM Ger Tue              132.5   139.4    0.0  16.3